In [ ]:
import matplotlib.pyplot as plt
import sys
import logging
sys.path.append('..')
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('napari').setLevel(logging.WARNING)
logging.getLogger('in_n_out').setLevel(logging.WARNING)
logging.getLogger('numcodecs').setLevel(logging.WARNING)
logging.getLogger('numba').setLevel(logging.WARNING)

from src import Receipt, load_data
from src.steps import (download_data, segment, detect_spots, get_cell_properties, 
                       return_data, match_masks, export_images, filter_csv, 
                       calculate_sharpness, reconcile_data)


### Initialize Receipt

In [ ]:
receipt = Receipt(
    analysis_name = 'default_name',
    nas_location = r'Users/Jack/_ImageDatasets/Sia/SS004_07212025_TroubleShooting/SS004_10min_100nM_7',
    local_location = None,
    data_loader = 'pycromanager_data_loader'
    )

## This is a formated dictionary that contains all the information on the 
# data to be processed and how to process it.
# Primary keys of interest:
#   meta_arguments - parameter relating to the data
#   steps - parameters of each step
#   step_order - order of the steps 
#   dirs - key directories 

# if nas_location is provided and local_location is not, a local location will be calculated in database dir
# the data_loader decribes which format your data is in. We will treat the file structure of your data
# as a data structures. 'pycromanager_data_loader' use the ndtiff and extends it so we can save result to the 
# directory

### Get the data

In [ ]:
dd = download_data(receipt, 'download_data')
receipt = dd.process()

## This downloads the data from a nas with connections established in config_cluster.yml
# It will download the data from the shared folder on the nas so
# Z:\Users\Jack\_ImageDatasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_5min_5uM_1 becomes
# Users\Jack\_ImageDatasets\IntronDiffusion\04102025_Triptolidetimeseries_JF001\JF001_5min_5uM_1 because 
# the Z drive on my computer refers to the shared directory on the nas

### Segmentation

In [ ]:
nuc_seg = segment(receipt, 'segment_nuc')
receipt = nuc_seg.gui()

## Uses cellpose 4 to segment cells and displays them in a gui
# pretrained_model_name: str = None - model name in models directory
# diameter: float = 180 - size of cells
# invert: bool = False - inverts image
# normalize: bool = True - normalizes cells from precentiles
# do_3D:bool=False, 
# min_size:float=500,
# flow_threshold:float=0, 
# cellprob_threshold:float=0, 

# -> 'mask_name'.tif in results dir

In [ ]:
match_cyto_and_nuc = match_masks(receipt, 'match_cyto_and_nuc')
receipt = match_cyto_and_nuc.process()

## matches any cytoplasmic and nuclear masks to have the same labels
# nuc_mask_name - name of nuc masks, 'nuc_masks' by default
# cyto_mask_name - name of cyto masks, 'cyto_masks' by default
# single_nuc - allows for lone nucleus, True by default
# cell_alone - allows for cells without nucleus, False by default

# -> rewrites masks in there tifs location on the drive using memmap(file_path, mode='r+')

### Detect Spots

In [ ]:
ds = detect_spots(receipt, 'detect_dusp1')
receipt = ds.gui()

## Uses big fish to detect spots 
# spot_name:str - name of the spots (i.e. dusp1)
# image - czyx shaped image
# FISHChannel: int - channel index with fish images
# voxel_size_yx: float - calculated from metadata (nm)
# voxel_size_z: float - calculated from metadata (nm)
# spot_yx: float - spot size in xy (nm)
# spot_z: float - spot size in z (nm)
# timepoint:int - give by iteration
# fov:int - given by iteration
# nucChannel: int = None - nuc channel index
# nuc_mask:np.array=None - zyx nuc mask from 'nuc_masks'
# cell_mask:np.array=None - zyx cyto mask from 'cyto_masks'
# threshold: Union[int, str] = None, 
# snr_threshold: float = None, 
# snr_ratio: float = None,
# alpha: float = 0.7, 
# beta:float = 1, 
# gamma:float = 5, 
# cluster_radius:int = 500, 
# min_num_spot_per_cluster:int = 4, 
# use_log_hook:bool = False, 
# verbose:bool = False, 
# display_plots: bool = False, 
# use_pca: bool = False,
# sub_pixel_fitting: bool = False, 
# minDistance:Union[float, list] = None,

# -> save 'spot_name'_cellresults.csv, 'spot_name'_spotresults.csv, 'spot_name'_clusterresulsts.csv


In [ ]:
receipt = reconcile_data(receipt, 'reconcile_spots')

## Recalculates cell results
# cell_key:str - cell props key
# spot_key:str - spot props key

# -> cellresults_reconciled_data.csv which will have all the cell props and RNA counts


### Get Properties

In [ ]:
measure_cell_props = get_cell_properties(receipt, 'measure_cell_props')
receipt = measure_cell_props.process()


## Calculates cell props 
# image, 
# cell_mask=None, 
# nuc_mask=None, 
# middle_zs:int = None - trims to middle 3 zs # TODO make this based on focal plane
# props_to_measure= ['label', 'bbox', 'area', 'centroid', 'intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std']

# -> cellproperties.csv with measurements made for every channel

In [ ]:
cs = calculate_sharpness(receipt, 'calc_sharpness')
receipt = cs.process({
    'channel': 0
})


## calculates the sharpness of every z in dataset
# zyx_image: np.array,
# sharpness_metric: Union[list,str]=None

# -> sharpnesses.json - dict of format [p][t][z][metric]

### Manipulate Data

In [ ]:
receipt = return_data(receipt, 'return_data').process()

## returns data to nas

In [ ]:
fc = filter_csv(receipt, 'filter_csv')
receipt = fc.gui()

## Filters csv based on set filters
# csv_to_filter - name of data to filter
# columns_to_filter_on - list of column names to filter on 
# values_to_filter_on - list of (min value, max value) to filter data on

# -> f'{csv_to_filter}_filtered.csv'

In [ ]:
receipt = export_images(receipt, 'export_tifs')

# Export images into different formats
# channel_order - channel order
# export_format - file format to export as

### Receipt

In [ ]:
import json

print('Meta Arguments: ')
meta_args_str = json.dumps(receipt['meta_arguments'], indent=4)
print(meta_args_str[1:-1])

print('Step Order: ')
for i, step in enumerate(receipt['step_order'], 1):
    print(f"\t{i}) {step}")

In [ ]:
receipt.save('new_pipeline.json')